In [156]:
import pdb
import pickle
import dill
import json
import sys
sys.path.append("../")

In [157]:
from training.constants import TACRED_LABEL_MAP

In [158]:
label_space = list(TACRED_LABEL_MAP.keys())[:-1]

In [159]:
with open("../data/training_data/parser.p", "rb") as f:
    parser = dill.load(f)

In [160]:
labeling_functions = parser.labeling_functions

In [161]:
len(labeling_functions)

161

In [162]:
ner_types = parser.ner_types

In [163]:
len(ner_types)

161

In [164]:
with open("../data/tacred_train.json") as f:
    train_data = json.load(f)

In [165]:
train_data[0]

{'text': 'OBJ-PERSON resigned in October last year to form the SUBJ-ORGANIZATION ( ABC ) , crossing the floor with 17 members of parliament , causing constitutional monarch King Letsie III to dissolve parliament and call the snap election .',
 'label': 'org:founded_by'}

In [166]:
with open("../training/train_phrases.p", "rb") as f:
    train_phrases = pickle.load(f)

In [167]:
len(train_phrases)

68124

In [168]:
matched_indicies = {}
multiple_matches = {}
for i, phrase in enumerate(train_phrases):
    not_matched = True
    for j, function in enumerate(labeling_functions):
        try:
            if function(phrase):
                subj_type = phrase.ners[phrase.subj_posi].lower()
                obj_type = phrase.ners[phrase.obj_posi].lower()
                if ner_types[function][0] == subj_type and ner_types[function][1] == obj_type:
                    if i not in matched_indicies:
                        matched_indicies[i] = (labeling_functions[function], j)
                    elif i not in multiple_matches:
                        multiple_matches[i] = [matched_indicies[i]]
                        multiple_matches[i].append((labeling_functions[function], j))
                    else:
                        mutliple_matches[i].append((labeling_functions[function], j))
        except:
            continue        

In [169]:
len(matched_indicies)

2745

In [170]:
len(multiple_matches)

67

In [171]:
sum([len(multiple_matches[key]) for key in multiple_matches])/len(multiple_matches)

2.0

In [172]:
annnoted_labels = []
true_labels = []
for key in matched_indicies:
    annnoted_labels.append(matched_indicies[key][0])
    true_labels.append(train_data[key]["label"])

In [173]:
annnoted_labels[:10]

['per:title',
 'org:alternate_names',
 'per:title',
 'per:title',
 'org:top_members/employees',
 'org:founded_by',
 'per:title',
 'org:top_members/employees',
 'org:alternate_names',
 'org:alternate_names']

In [174]:
true_labels[:10]

['per:title',
 'org:alternate_names',
 'per:title',
 'per:title',
 'org:top_members/employees',
 'org:founded_by',
 'no_relation',
 'org:top_members/employees',
 'no_relation',
 'org:alternate_names']

In [175]:
import sklearn.metrics as sklm

In [176]:
sklm.precision_recall_fscore_support(true_labels, annnoted_labels, average="micro", labels=label_space)

(0.8564663023679417, 0.9857442348008386, 0.9165692007797271, None)

In [55]:
with open("../../../ziqi/NExT/tacred_labeled.json") as f:
    ziqi_index = json.load(f)

In [59]:
missing_count = 0
for pair in ziqi_index:
    if pair[0] not in matched_indicies:
        missing_count += 1

In [61]:
missing_count

24

In [62]:
ziqi_labels = []
ziqi_true_labels = []
for pair in ziqi_index:
    ziqi_labels.append(pair[1])
    ziqi_true_labels.append(train_data[pair[0]]["label"])

In [71]:
sklm.precision_recall_fscore_support(ziqi_true_labels, ziqi_labels, average="micro", labels=label_space)

(0.8565137614678899, 0.9856418918918919, 0.9165521303750246, None)

In [113]:
matched_indicies_per_function = {}
for i, phrase in enumerate(train_phrases):
    not_matched = True
    for j, function in enumerate(labeling_functions):
        try:
            if function(phrase):
                if j not in matched_indicies_per_function:
                    matched_indicies_per_function[j] = {}
                if i not in matched_indicies_per_function[j]:
                    matched_indicies_per_function[j][i] = (labeling_functions[function], j)
        except:
            continue        

In [116]:
for key in matched_indicies_per_function:
    print(key, (len(matched_indicies_per_function[key])))

6 1538
13 870
1 98
3 507
10 22
29 83
23 64
7 51
15 70
0 41
18 29
16 80
5 96
27 56
2 20
20 16
19 46
22 21
32 42
25 34
36 30
21 13
35 32
24 14
30 19
12 14
4 25
31 19
34 24
9 13
33 23
26 13
14 24
28 30
37 19
11 21
8 15
17 11


In [119]:
functions = list(labeling_functions.keys())

In [130]:
labeling_functions[functions[3]]

'org:country_of_headquarters'

In [121]:
semantic_reps = parser.semantic_reps

In [129]:
for key in semantic_reps:
    if semantic_reps[key] == functions[3]:
        print(key)

('.root', ('@And', ('@Is', ('@Word', "'s"), ('@between', ('@And', 'ArgY', 'ArgX'))), ('@Is', 'There', ('@AtMost', ('@between', ('@And', 'ArgY', 'ArgX')), ('@Num', '2', 'tokens')))))


In [131]:
parser.filtered_raw_explanations[('.root', ('@And', ('@Is', ('@Word', "'s"), ('@between', ('@And', 'ArgY', 'ArgX'))), ('@Is', 'There', ('@AtMost', ('@between', ('@And', 'ArgY', 'ArgX')), ('@Num', '2', 'tokens')))))]

'There are no more than two words between SUBJ and OBJ and the word "\'s" occurs between SUBJ and OBJ'